### Imports


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import LeaveOneGroupOut, KFold
from sklearn.preprocessing import LabelEncoder
from torchvision.models import mobilenet_v3_small
from tqdm import tqdm

from src.dataset_loaders import ISAdetectDataset
from src.transforms import GrayScaleImage

### Setup


In [2]:
# Specify the model
MODEL = mobilenet_v3_small
TARGET_FEATURE = "architecture"

# Model hyperparameters
BATCH_SIZE = 64
NUM_EPOCHS = 1
LEARNING_RATE = 1e-4

# Specify which groups to use as validation set. Set to None to validate all groups.
VALIDATION_GROUPS = None
# VALIDATION_GROUPS = ["mips", "mipsel"]

# Set to an integer to limit the dataset size. Set to None to disable limit.
MAX_FILES_PER_ISA = None

### Helper functions


In [3]:
def get_device():
    """
    Returns 'cuda' if CUDA is available, else 'mps' if Apple Silicon GPU is available,
    otherwise 'cpu'.
    """
    device = None
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    print(f"Using device: {device}")
    return device

### Prepare


In [4]:
device = get_device()

dataset = ISAdetectDataset(
    dataset_path="../../dataset/ISAdetect/ISAdetect_full_dataset",
    transform=GrayScaleImage(224, 224),
    file_byte_read_limit=224 * 224,
    per_architecture_limit=MAX_FILES_PER_ISA,
)

groups = list(map(lambda x: x["architecture"], dataset.metadata))
target_feature = list(map(lambda x: x[TARGET_FEATURE], dataset.metadata))
print(f"groups: {set(groups)}")
print(f"features: {set(target_feature)}")

Using device: cuda
groups: {'powerpcspe', 's390', 's390x', 'armhf', 'ppc64el', 'ia64', 'x32', 'powerpc', 'i386', 'mips', 'amd64', 'ppc64', 'mips64el', 'arm64', 'hppa', 'sparc', 'm68k', 'riscv64', 'sparc64', 'mipsel', 'alpha', 'armel', 'sh4'}
features: {'powerpcspe', 's390', 's390x', 'armhf', 'ppc64el', 'ia64', 'x32', 'powerpc', 'i386', 'mips', 'amd64', 'ppc64', 'mips64el', 'arm64', 'hppa', 'sparc', 'm68k', 'riscv64', 'sparc64', 'mipsel', 'alpha', 'armel', 'sh4'}


### Train and evaluate


In [5]:
kfold = KFold(n_splits=10, shuffle=True)
label_encoder = LabelEncoder()

fold = 1
accuracies = {}
corrects = {}
totals = {}
for train_idx, test_idx in kfold.split(
    X=range(len(dataset))
):

    print(f"\n=== Fold {fold} ===")
    fold += 1

    all_train_labels = [dataset.metadata[i][TARGET_FEATURE] for i in train_idx]
    label_encoder.fit(all_train_labels)

    train_dataset = Subset(dataset, train_idx)
    test_dataset = Subset(dataset, test_idx)

    train_loader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8
    )
    test_loader = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8
    )

    model = MODEL(num_classes=23, weights=None)
    model.features[0][0] = nn.Conv2d(
        1, 16, kernel_size=3, stride=2, padding=1, bias=False
    )

    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # Train model
    for epoch in range(NUM_EPOCHS):
        model.train()
        print(f"Epoch {epoch+1}:")

        for images, labels in tqdm(train_loader):
            images = images.to(device)

            encoded_labels = torch.from_numpy(
                label_encoder.transform(labels[TARGET_FEATURE])
            ).to(device)

            optimizer.zero_grad()
            predictions = model(images)
            loss = criterion(predictions, encoded_labels)
            loss.backward()
            optimizer.step()

        # Evaluate model
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                encoded_labels = torch.from_numpy(
                    label_encoder.transform(labels[TARGET_FEATURE])
                ).to(device)

                outputs = model(images)

                _, predicted = torch.max(outputs, 1)
                correct += (predicted == encoded_labels).sum().item()
                total += encoded_labels.size(0)

                for i in range(len(predicted)):
                    # add accuracy per architecture
                    arch = label_encoder.inverse_transform([encoded_labels[i].item()])[0]
                    if predicted[i] == encoded_labels[i]:
                        corrects[arch] = corrects.get(arch, 0) + 1
                    totals[arch] = totals.get(arch, 0) + 1

        accuracy = correct / total
        print(f"Test Accuracy: {100*accuracy:.2f}%")

    accuracies[fold] = accuracy


=== Fold 1 ===
Epoch 1:


100%|██████████| 1356/1356 [00:45<00:00, 30.07it/s]


Test Accuracy: 85.57%

=== Fold 2 ===
Epoch 1:


100%|██████████| 1356/1356 [00:44<00:00, 30.28it/s]


Test Accuracy: 84.85%

=== Fold 3 ===
Epoch 1:


...

Test Accuracy: 86.19%

=== Fold 9 ===
Epoch 1:


100%|██████████| 1356/1356 [00:45<00:00, 29.58it/s]


Test Accuracy: 87.19%

=== Fold 10 ===
Epoch 1:


100%|██████████| 1356/1356 [00:45<00:00, 29.86it/s]


Test Accuracy: 86.21%


### Evaluate


In [7]:
print("Test accuracies for each architecture:")
for arch, correct_count in corrects.items():
    print(f"{arch}: {100*correct_count/totals[arch]:.2f}%")


# Print overall performance across folds
mean_acc = np.mean(list(accuracies.values()))
std_acc = np.std(list(accuracies.values()))
print(f"\nAverage LOGO cross-validated test accuracy: {mean_acc:.4f} ± {std_acc:.4f}")

Test accuracies for each architecture:
alpha: 97.33%
mips: 92.82%
sh4: 91.08%
armel: 97.42%
powerpcspe: 60.74%
sparc64: 66.30%
x32: 58.43%
sparc: 80.26%
powerpc: 65.05%
ia64: 98.46%
i386: 75.55%
hppa: 96.83%
mipsel: 93.17%
m68k: 91.50%
ppc64: 92.76%
armhf: 91.34%
s390x: 96.38%
ppc64el: 95.66%
mips64el: 94.64%
riscv64: 87.66%
arm64: 97.08%
s390: 95.33%
amd64: 43.63%

Average LOGO cross-validated test accuracy: 0.8530 ± 0.0270
